# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [3]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [4]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

### Fit a Model

Fit a linear regression model on the training set

In [6]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [7]:
# Instantiate and fit a linear regression model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

### Calculate MSE

Calculate the mean squared error on the test set

In [8]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

In [9]:
# Calculate MSE on test set
test_mse = mean_squared_error(y_test, linreg.predict(X_test))
print('Test MSE: ', test_mse)

Test MSE:  0.20440212309132563


## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [10]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score


In [14]:
# Find MSE scores for a 5-fold cross-validation
cross_val_score(linreg, X, y, cv=5)

array([0.86724871, 0.82916275, 0.82017954, 0.81167639, 0.78362276])

In [15]:
# Get the average MSE score
np.mean(cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error"))

-0.1770283421000112

Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [16]:
# Your answer here
print('I trust the cross val method more, as it averages over the noise from a individual train/test splits.')

I trust the cross val method more, as it averages over the noise from a individual train/test splits.


## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [19]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


In [23]:
np.floor(example_data.shape[0]/3)

2.0

`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [69]:
def kfolds(data, k):
    folds = []
    
    # Your code here
    rem = np.mod(data.shape[0], k) # get remainder
    n_elem = int(np.floor(data.shape[0]/k)) # number of elements in fold (or 1 more if extra)
  
    fold_start = 0
    for i in range(k): #for each fold
        if i < rem: #add 1 to number of elements to pull
            if isinstance(data, pd.Series):
                thisfold = data.iloc[(fold_start):(fold_start+n_elem+1)]
            else:
                thisfold = data.iloc[(fold_start):(fold_start+n_elem+1), :]
                
            fold_start = fold_start + n_elem + 1
        else: #just pull n elements
            if isinstance(data, pd.Series):
                thisfold = data.iloc[(fold_start):(fold_start+n_elem)]
            else:
                thisfold = data.iloc[(fold_start):(fold_start+n_elem), :]
                
            fold_start = fold_start + n_elem
            
        folds.append(thisfold)
    return folds

In [70]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange
2  yellow 

   color
3  green
4   blue 

    color
5  indigo
6  violet 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [71]:
# Apply kfolds() to ames_data with 5 folds
X_folds = kfolds(X, 5)
y_folds = kfolds(y, 5)
X_folds, y_folds


([     LotArea_log  1stFlrSF_log  GrLivArea_log  BldgType_2fmCon  \
  0      -0.133185     -0.803295       0.529078                0   
  1       0.113403      0.418442      -0.381715                0   
  2       0.419917     -0.576363       0.659449                0   
  3       0.103311     -0.439137       0.541326                0   
  4       0.878108      0.112229       1.281751                0   
  ..           ...           ...            ...              ...   
  287    -0.208982     -0.795950      -1.538509                0   
  288     0.156994     -0.645537      -1.395230                0   
  289    -0.070186     -1.445511      -0.079173                0   
  290     1.053039     -0.074628       0.874786                0   
  291    -0.898448     -0.522097       0.539579                1   
  
       BldgType_Duplex  BldgType_Twnhs  BldgType_TwnhsE  KitchenQual_Fa  \
  0                  0               0                0               0   
  1                  0         

### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [80]:
X_folds = kfolds(X, 5)
temp = X_folds.pop(1)
temp

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
292,0.447039,0.120464,0.539579,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
293,1.178606,0.894327,1.369125,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
294,0.113403,1.250703,0.411072,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
295,-0.254224,-0.304506,-1.070374,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
296,0.802095,-0.610760,0.249989,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,0.568649,-0.160375,0.674518,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
580,0.921659,0.809582,-0.009127,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
581,0.654817,1.933035,1.061041,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
582,0.518864,-0.953914,-1.688980,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [78]:
len(X_folds)

4

In [88]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = kfolds(X, k)
    X_test = X_train.pop(n)
    X_train = pd.concat(X_train, axis=0)
    
    y_train = kfolds(y, k)
    y_test = y_train.pop(n)
    y_train = pd.concat(y_train, axis=0)
    
    #print(X_train.shape)
    #print(y_train.shape)
    
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    # Evaluate test errors
    test_errs.append(mean_squared_error(y_test, linreg.predict(X_test)))

print(np.mean(test_errs))

0.1770283421000112


If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [91]:
# Compare your results with sklearn results
from sklearn.model_selection import cross_val_score

print(np.mean(cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")))

-0.1770283421000112


This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.